# Main Focus of the Notebook: Running Laptrack on detections

### This notebook takes as input the dataframe which contains filtered detected spots and runs laptrack module on it. 
### A new dataframe containing tree id and track id is returned. For us main variable for tracks is track id. 

In [1]:
from os import path
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from laptrack import LapTrack, ParallelBackend
import napari
from skimage import io
import time  
import os 
import sys
import zarr 
import dask.array as da

### Do not change the code in the cell below

In [2]:
# This assumes that your notebook is inside 'Jupyter Notebooks', which is at the same level as 'test_data'
base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'movie_data')

zarr_directory = 'zarr_file/all_channels_data'
zarr_full_path = os.path.join(base_dir, zarr_directory)

input_directory = 'datasets'
input_file_name = 'cleaned_spots_intensities_c3_all.pkl'
input_directory_full = os.path.join(base_dir,input_directory, input_file_name)

output_directory = 'datasets'
output_file_name = 'track_df_c3_cleaned.pkl'
output_directory_full = os.path.join(base_dir,output_directory, output_file_name)

In [3]:
cleaned_spots_df = pd.read_pickle(input_directory_full)
z2 = zarr.open(zarr_full_path, mode='r')

## Parameter to optimize 

1. **max_distance:** The cost cutoff for the connected points in the track.

In [4]:
max_distance = 3
lt = LapTrack(
    track_dist_metric="sqeuclidean",  # The similarity metric for particles. See `scipy.spatial.distance.cdist` for allowed values.
    # the square of the cutoff distance for the "sqeuclidean" metric
    track_cost_cutoff=max_distance**2,
    gap_closing_cost_cutoff = (2*max_distance)**2,
    gap_closing_max_frame_count = 1,
    splitting_dist_metric="sqeuclidean",
    splitting_cost_cutoff=False,  # Enable splitting with a cost cutoff
    merging_dist_metric="sqeuclidean",
    merging_cost_cutoff=False  # Enable merging with a cost cutoff
)

In [5]:
start_time = time.time()

track_df, split_df, merge_df = lt.predict_dataframe(
    cleaned_spots_df,
    coordinate_cols=[
        "mu_x",
        "mu_y",
        "mu_z"
    ],  # the column names for the coordinates
    frame_col="frame",  # the column name for the frame (default "frame")
    only_coordinate_cols=False,  # if False, returned track_df includes columns not in coordinate_cols.
    # False will be the default in the major release.
)

end_time = time.time()

execution_time = end_time - start_time
print(f"Total execution time: {execution_time} seconds")
track_df = track_df.reset_index()

Total execution time: 105.13887619972229 seconds


### You can visualise the performance of the tracking algorithm in napari. If you feel adjustment is needed for the tracking parameters, you can adjust the parameters mentioned in the start accordingly. 

Once you open napari adjust the contrast so that spots are clearly visible 

Possible Adjustment Examples

1. If a single track appears to be broken into multiple tracks you can increase the max_distance and rerun tracking 
2. If multiple tracks appear to be merged into a single track you can decrease the max_distance and rerun tracking 

In [8]:
# Create a napari viewer
viewer = napari.Viewer()

#access channel 3 only from zarr array 
dask_array = da.from_zarr(z2)

#the axis arrangement is (t,c,z,y,x)
#for the sake of improved performance only 1 channel could be imported here (if images get super large and performance issues occur)
all_channels = dask_array[:,:,:,4:2050,:]

# Add the 4D stack to the viewer
layer_raw = viewer.add_image(all_channels, channel_axis = 1, name = ['channel 1', 'channel 2', 'channel 3'])
viewer.add_tracks(track_df[["track_id", "frame", "mu_z", "mu_y", "mu_x"]], name = 'all tracks')
#other useful parameters 
#color_map = list
#contrast_limits = list of list 

# Add Bounding Box
layer_raw[0].bounding_box.visible = True
layer_raw[1].bounding_box.visible = True
layer_raw[2].bounding_box.visible = True


In [ ]:
# Save the DataFrame to a pickle file
track_df.to_pickle(output_directory_full)